# Mapping!!!

Today, we're going to practice making a map in Python - it won't be as pretty as we necessarily want it to be, but it will help us build our Python skills and allow us to see the relationship between our ACS, Enviroscreen, and other datafiles and the the mapping functions in GeoPandas. 

As I was teaching myself how to map in Python, I found this tutorial particularly helpful!
https://www.earthdatascience.org/courses/earth-analytics-python/spatial-data-vector-shapefiles/intro-vector-data-python/

As always, we'll import our libraries.  You'll see more libraries than before - especially geopandas!

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt

## 1.  Bringing Data into a GIS Environment

### 1.1  Importing our Geographic Layer File

The first step is to import our geographic data - this can be a polygon, point, or line vector datafile.  We're going to use a "shapefile" of census tracts that I downloaded from the US Census Bureau for California.  All of the Census shapefiles for census geographies can be found here: 

https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.2010.html

Take a look at the folder you just uploaded.  How many files are there with the same name?  A shapefile is made up of a lot of different components - make sure that when you download a shapefile, you save all these components into the same folder.

We're going to import this shapefile as "tracts".  Note that rather than "pd." - which we called to bring in our .csv files into a dataframe, we're going to call "gpd.", which creates a **geo**dataframe.

To help me keep my data well organized, I'm going to add the suffix **gdf** to my filenames, just the way I used to add **df** to my dataframes. 

In [ ]:
tracts_gdf = gpd.read_file('cb_2018_06_tract_500k.shp')

### 1.2  Assessing Our Spatial Data

Before jumping right into mapping, it's always helpful to look at the data and take note of the column names, and make sure everything imported correctly.  

In [ ]:
print(type(tracts_gdf))
tracts_gdf.head()

### 1.3 Mapping!

So far, so good.  Let's map it!  Because "tracts" is a geodatabase, a simple "tracts.plot()" will do!

In [ ]:
tracts_gdf.plot()

### 1.4  But, I want San Francisco County!

Amazingly, we can treat our geodataframe just as we would a regular dataframe.  In the next cell, I've written code that tells Python to drop any tract that is not 075 (the fips code for San Francisco county).  

In [ ]:
tracts_gdf.drop(tracts_gdf[tracts_gdf['COUNTYFP']!="075"].index, inplace=True )
tracts_gdf.plot()

Let's get rid of the Farrallon islands and treasure island so our map looks better!

In [ ]:
tracts_gdf.drop(tracts_gdf[tracts_gdf['GEOID']=="06075980401"].index, inplace=True )
tracts_gdf.drop(tracts_gdf[tracts_gdf['GEOID']=="06075017902"].index, inplace=True )

Let's plot it again to check our work.

In [ ]:
tracts_gdf.plot()

### 1.5  Adding Data

It's a blue blob!!  What I really want to map is see the distribution of rent burdens in the city in 2017, and layer eviction notices on top to see if there's any visual pattern between where residents are more rent burdened and subsequent evictions.  I need to "join" my attribute data (rent burden) from the ACS and add those columns to the geodataframe. The most important thing to watch out for is the "key" variable that I want to merge on - I need to make sure they match exactly, so I'm going to make sure that Python brings in my FIPS code from my ACS data with the leading zero, since the geodataframe above shows a leading zero in the FIPS code.

In [ ]:
variable_types = {'fips': 'str'}
rentburden_df = pd.read_csv("RentBurden2000_2017.csv", dtype=variable_types)

In [ ]:
#let's make sure it looks ok!
rentburden_df.head()

In [ ]:
#now I can join, based on the matching key variables
#We want to make sure that we are joining the data to the geodataframe
merged_gdf=tracts_gdf.set_index("GEOID").join(rentburden_df.set_index("fips"))
merged_gdf

That was a lot easier than in ArcGIS.

Before we add in our eviction data, let's take a quick look at our new geodataframe ("merged") and quickly map the distribution of cost burdens in the city.

In [ ]:
#note that it's exactly like the previous code, though I've added the column I want to plot
#I also specified the size of the figure
merged_gdf.plot(column='pct_costburdened_2017', figsize=(14,10))

### 1.6  Adding Lat Long Data from a CSV

Ok - I know.  San Francisco is still squished.  But let's add our eviction data before we start playing with our map design. I have created a .csv with eviction data that includes the longitude and latitude for each eviction that has happened since 2011.  I'm going to read it in with pandas and create a dataframe with the data. It's not a geodataframe yet, since I haven't told python that the longitude and latitude columns represent spatial data.

In [ ]:
evictions_df=pd.read_csv("SFEvictions.csv")
evictions_df.head()

### 1.7  Bringing in XY Coordinates

In order to bring in my .csv file as a geodataframe, I need to know what coordinate reference system it's in.  Unfortunately, this was not part of the metadata provided with the dataset.  But I can see that the lat/long coordinates match the map above.  So I'm going to take a guess and assume that the data are in the same reference system.  I can request Python to give me the coordinate reference system for my merged_gdf.

In [ ]:
merged_gdf.crs

Let's give that a try!  The code for reading a .csv file into a geodataframe is as follows.  In the first line, I'm specifying the coordinate reference system, and in the second line, I'm specifying the columns that Python should read as the geometry for the geographic data.  The third line creates the geodataframe, calling the dataset, the crs, and the geometry.

In [ ]:
crs={'init':'epsg:4269'}
geometry = [Point(xy) for xy in zip(evictions_df.Longitude, evictions_df.Latitude)]
evictions_gdf = GeoDataFrame(evictions_df, crs=crs, geometry=geometry)

Let's take a look!!!  I can now specify my base layer as the merged_gdf geodataframe, and the evictions geodataframe as
what I want to plot on the axis on top of the base.  Note I've also added an option for a better color ramp!

In [ ]:
base = merged_gdf.plot(column='pct_costburdened_2017', cmap='OrRd', figsize=(14,10))
evictions_gdf.plot(ax=base, markersize=1)

## 2  Making the Map Look Better

### 2.1  Customizing our Map

While geopandas can help us make quick maps using the simple commands above, it's nice to be able to customize them more. 

Let's look at the cell below.

The first row creates a figure (named "figure"), with one axis.  This figure is called by using the command plt.subplots (part of the library matplotlib); basically, it is creating a "frame" to hold our map - we can actually specify multiple subplots to put figures side by side, but let's not get ahead of ourselves! Note how the code is returning two objects with different names (figure and ax) by  listing them at the front of the line, separated by commas.

Second, we plot the geographies, but this time we tell the function that we want it to draw the polygons on the axes we are passing, ax. 

The size of the plot is changed equally easily in this context. The only difference is that it is specified when we create the figure with the argument figsize. The first number represents the width, the X axis, and the second corresponds with the height, the Y axis. On the third line, we tell Python to preserve the ratio, and in the fourth line we effectively remove the box with coordinates.

We add a title.

Finally, we draw the entire plot by calling plt.show().

In [ ]:
figure, ax = plt.subplots(figsize=(14,10))
ax = merged_gdf.plot(column="pct_costburdened_2017", legend=True, ax=ax, cmap="Blues")
lims=plt.axis("equal")
ax.set_axis_off()

ax.set_title('San Francisco Rent Burden, 2017')

plt.show()

There are lots of color options:
https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
figure, ax = plt.subplots(figsize=(14,10))
base = merged_gdf.plot(column="pct_costburdened_2017", legend=True, ax=ax, cmap="Blues")
evictions_gdf.plot(ax=base, markersize=1, color="Black")
lims=plt.axis("equal")
ax.set_axis_off()

ax.set_title('San Francisco Rent Burden, 2017', fontdict= 
            {'fontsize':25})

plt.show()

### 2.2 Changing the Projection

San Francisco is still squished.  Let's change the projection!  To do this, we are going to identify the right projection for the San Francisco Bay Area, doing lots and lots of googling.  Then we're going to take Carolina's word for it and assign our layers a crs of epsg:2227.

In [ ]:
#let's check our "starting" projections
print(evictions_gdf.crs)
print(merged_gdf.crs)

In [ ]:
#I'm going to create a new object called merged_proj so I know I've projected the layer
merged_proj=merged_gdf.to_crs({'init' : 'epsg:2227'})

In [ ]:
#It turns out reprojecting data takes a long time, especially with a big datafile. 
#To make it quicker, I'm going to focus on the association between rent burdens in 2017 and 
#evictions in 2019 due to nonpayment of rent.
evictions_gdf.drop(evictions_gdf[evictions_gdf['Year']==2019].index, inplace=True )
evictions_gdf.drop(evictions_gdf[evictions_gdf['Non Payment']==0].index, inplace=True)
evictions_gdf

In [ ]:
#then, I'm going to reproject evictions to match the crs of my merged base layer
#This is likely to take a little while - chat with your neighbor!
evictions_proj=evictions_gdf.to_crs(merged_proj.crs)

In [ ]:
#checking to see if the bounds and metrics of my two projected layers are the same!
print(evictions_proj.total_bounds)
print(merged_proj.total_bounds)

In [ ]:
figure, ax = plt.subplots(figsize=(14,10))
base = merged_proj.plot(column="pct_costburdened_2017", legend=True, ax=ax, cmap='OrRd')
evictions_proj.plot(ax=base, markersize=1, color="Black")
lims=plt.axis("equal")
ax.set_axis_off()

ax.set_title('San Francisco Rent Burden, 2017', fontdict= 
            {'fontsize':25})

plt.show()

## Yipeee!!!!  I have an unsquished SF map!!!

Now, customize it in a way that looks good to you!  When you're ready to export it to put it in your case study, you just need to run the cell below.

In [ ]:
figure.savefig('RentBurdenEvictions.png')

## Some Fun Bonus Material

### Bonus 1: Let's make a heat map of evictions!

In [ ]:
def heatmap(d, bins=(100,100), smoothing=1.3, cmap='jet'):
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]

    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.filters.gaussian_filter(logheatmap, smoothing, mode='nearest')
    #return logheatmap
    merged_gdf.plot(color='none', edgecolor='white', linewidth=.5, alpha=.5, figsize=(14,10))

    plt.imshow(logheatmap, cmap=cmap, extent=extent)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

heatmap(evictions_gdf, bins=100, smoothing=3)

### Bonus 2: Let's create a hover tool that shows the percent rent burdened in each tract

In [ ]:
from bokeh.io import curdoc, output_notebook
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

In [ ]:
import json
merged_json = json.loads(merged_proj.to_json())
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]

#Reverse color order so that dark blue is highest value.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 1, nan_color = '#d9d9d9')

#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Add hover tool
hover = HoverTool(tooltips = [ (('pct_costburdened_2017', '@pct_costburdened_2017'))])

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     location = (0,0), orientation = 'horizontal')

#Create figure object.
p = figure(title = 'Percent Cost Burdened, 2017', plot_height = 950 , plot_width = 950, toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'pct_costburdened_2017', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify layout
p.add_layout(color_bar, 'below')

output_notebook()

show(p)
